# Parameter tables

This notebook explains how the parameter table works. With this table a sweep object keeps track of which parameters there are and how they are dependent on each other. We also keep track of inferred parameters through this table 

In [40]:
import numpy as np
import matplotlib.pyplot as plt

from qcodes import new_experiment
from qcodes.dataset.plotting import plot_by_id
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep import sweep, chain, nest, setter

In [29]:
x = ManualParameter("x", unit="V")
y = ManualParameter("y", unit="I")

m = ManualParameter("m", unit="R")
m.get = lambda:np.sin(x())

n = ManualParameter("n", unit="R")
n.get = lambda:np.sin(x()**2 + y())

Let us see how the parameters table of a simple sweep object looks like 

In [30]:
sweep(x, np.linspace(-4, 4, 100)).parameter_table._table_list

[{'dependent_parameters': [], 'independent_parameters': [('x', 'V')]}]

We see that internally, the parameters table is composed out of a list of dictionaries. All dependent parameters in this dictionary are dependent on all independent parameters of the same dictionary 

Next, we will nest a measurement in the sweep objects and observe how the table changes 

In [31]:
nest(sweep(x, np.linspace(-4, 4, 100)), m).parameter_table._table_list

[{'dependent_parameters': [('m', 'R')],
  'independent_parameters': [('x', 'V')]}]

We see that we have introduced a dependent parameter by nesting a measurement in a simple sweep. Let us nest two simple sweeps...

In [32]:
nest(
    sweep(x, np.linspace(-4, 4, 100)), 
    sweep(y, np.linspace(-4, 4, 100))
).parameter_table._table_list

[{'dependent_parameters': [],
  'independent_parameters': [('x', 'V'), ('y', 'I')]}]

In [33]:
nest(
    sweep(x, np.linspace(-4, 4, 100)), 
    sweep(y, np.linspace(-4, 4, 100)), 
    m 
).parameter_table._table_list

[{'dependent_parameters': [('m', 'R')],
  'independent_parameters': [('x', 'V'), ('y', 'I')]}]

We see that now "m" is dependent on both "x" and "y". Lets see what effect chaining has 

In [34]:
chain(
    sweep(x, np.linspace(-4, 4, 100)), 
    sweep(y, np.linspace(-4, 4, 100))
).parameter_table._table_list

[{'dependent_parameters': [], 'independent_parameters': [('x', 'V')]},
 {'dependent_parameters': [], 'independent_parameters': [('y', 'I')]}]

In [36]:
chain(m, n).parameter_table._table_list

[{'dependent_parameters': [('m', 'R')], 'independent_parameters': []},
 {'dependent_parameters': [('n', 'R')], 'independent_parameters': []}]

Chaining has the effect of creating two subtables. If we consider the nesting operation to be equivalent to a multiplication and chaining as equivalent to summation, then the [distributive property](https://en.wikipedia.org/wiki/Distributive_property) applies to the parameter table. That is: 

```
table1 * (table2 + table3) = table1 * table2 + table1 * table 3
```

In [37]:
nest(
    sweep(x, np.linspace(-4, 4, 100)), 
    chain(m, n)  
).parameter_table._table_list

[{'dependent_parameters': [('m', 'R')],
  'independent_parameters': [('x', 'V')]},
 {'dependent_parameters': [('n', 'R')],
  'independent_parameters': [('x', 'V')]}]

This is equivelent to 
```python
for x in np.linspace(-4, 4, 100): 
    m()
    n()
```

In [38]:
chain(
    nest(sweep(x, np.linspace(-4, 4, 100)), m), 
    nest(sweep(x, np.linspace(-4, 4, 100)), n)
).parameter_table._table_list

[{'dependent_parameters': [('m', 'R')],
  'independent_parameters': [('x', 'V')]},
 {'dependent_parameters': [('n', 'R')],
  'independent_parameters': [('x', 'V')]}]

This is equivelent to 
```python
for x in np.linspace(-4, 4, 100):
    m()
    
for x in np.linspace(-4, 4, 100):
    n()
```

We see that the distributive property holds and that in both instances we have created parameter "m" which only depends on "x" and another parameter "n" which likewise only depends on "x"

## inferred parameters

In this section we discuss how the parameters table deals with inferred parameters. Let's create one first...

In [41]:
@setter([("x", "V")], inferred_parameters=[("x_mv", "mV")])
def mysetter(xv): 
    return xv * 1000

In [42]:
x_setpoints = np.linspace(-4, 4, 100)

sweep_object = sweep(mysetter, x_setpoints)(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

In [43]:
sweep_object.parameter_table._table_list

[{'dependent_parameters': [('m', 'R')],
  'independent_parameters': [('x', 'V'), ('x_mv', 'mV')]},
 {'dependent_parameters': [('n', 'R')],
  'independent_parameters': [('x', 'V'), ('x_mv', 'mV'), ('y', 'I')]}]

We see that both the original parameter and the inferred parameter are listed as independent. A separate dictionary in the parameter table keeps track of inferred parameters 

In [44]:
sweep_object.parameter_table.inferred_from_dict

{'x_mv': ['x']}

Note that when we create the paramspecs, we will register the dependent parameters "m" and "n" to only depend on "x_mv" and *not* on both "x" and "x_mv". We will register "x_mv" as being inferred from "x"